In [1]:
from google.colab import drive
drive.mount('/content/drive')  

Mounted at /content/drive


In [2]:
%cd  /content/drive/MyDrive/Colab_Notebooks/comp_bio

/content/drive/MyDrive/Colab_Notebooks/comp_bio


In [ ]:
# Install rdkit
%%bash
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
conda config --set always_yes yes --set changeps1 no
conda install -q -y -c conda-forge python=3.7
conda install -q -y -c conda-forge rdkit==2020.09.2 

In [4]:
# Install rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

try:
  from rdkit import Chem
  from rdkit.Chem.Draw import IPythonConsole
except ImportError:
  print('Stopping RUNTIME. Colaboratory will restart automatically. Please run again.')
  exit()

Stopping RUNTIME. Colaboratory will restart automatically. Please run again.


In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors

- Data for the protein is taken from "dataset/celegans_uniprot.csv" (human protein sequences in uniprot). With word2vec.py it is converted to an embedding. 
- In mol_featurizer.py is loaded "../data/GPCR_train.txt" where it reads the smile format. 
1. From GPCR_train the smiles, sequence, **interaction** are obtained. 
2. Mol_features(smiles) returns **compounds**(which is atom_feature) and **adjacencies**
3. **Proteins** are obtained from the embeddings and sequences
- All the npy compounds, adjacencies, proteins, interactions are saved

Data (13022 examples):
- Compounds: list with all the molucules (mol.GetNumAtoms(), num_atom_feat=34) 
- Adjacencies: Adjacency matrix of each molecule (AdjacencyMatrix(mol),Chem.GetAdjacencyMatrix(mol))
- Proteins: for each protein, get protein embedding,infer a list of 3-mers to (num_word,100) matrix. For each sequence, use word2vec to obtain the data

The data is in GPCR_train, each element is a compound, a protein (sequence), and the interaction from the compound and the protein. 
1. From the compound it takes the compound.npy, which is consists of, for each compound, all the atoms it contains and for each atom all its features. Therefore, in compounds, for each compound it has a shape e.g. [31,34], which is 31 atoms and 34 features for each atom
2. From the compound it also takes the adjacency matrix, saved as adjacencies.npy, which reflects the position of each atom, and whether it is bonded with another atom (value 1) or not (value 0).
3. Then, from the file it also takes sequences, which refers to the protein that interact with that said compound. It applies word2vec and list of 3-mers to obtain a (num_word,100) matrix for each protein (sequence), where num_word is the number of words in the sequence.
4. Interaction: whether the compound or molecule interacts with the protein or not.


In [ ]:
num_atom_feat = 34
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return [x == s for s in allowable_set]


def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return [x == s for s in allowable_set]


def atom_features(atom,explicit_H=False,use_chirality=True):
    """Generate atom features including atom symbol(10),degree(7),formal charge,
    radical electrons,hybridization(6),aromatic(1),Chirality(3)
    """
    symbol = ['C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Br', 'I', 'other']  # 10-dim
    degree = [0, 1, 2, 3, 4, 5, 6]  # 7-dim
    hybridizationType = [Chem.rdchem.HybridizationType.SP,
                              Chem.rdchem.HybridizationType.SP2,
                              Chem.rdchem.HybridizationType.SP3,
                              Chem.rdchem.HybridizationType.SP3D,
                              Chem.rdchem.HybridizationType.SP3D2,
                              'other']   # 6-dim
    results = one_of_k_encoding_unk(atom.GetSymbol(),symbol) + \
                  one_of_k_encoding(atom.GetDegree(),degree) + \
                  [atom.GetFormalCharge(), atom.GetNumRadicalElectrons()] + \
                  one_of_k_encoding_unk(atom.GetHybridization(), hybridizationType) + [atom.GetIsAromatic()]  # 10+7+2+6+1=26

    # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
    if not explicit_H:
        results = results + one_of_k_encoding_unk(atom.GetTotalNumHs(),
                                                      [0, 1, 2, 3, 4])   # 26+5=31
    if use_chirality:
        try:
            results = results + one_of_k_encoding_unk(
                    atom.GetProp('_CIPCode'),
                    ['R', 'S']) + [atom.HasProp('_ChiralityPossible')]
        except:
            results = results + [False, False] + [atom.HasProp('_ChiralityPossible')]  # 31+3 =34
    return results


def adjacent_matrix(mol):
    adjacency = Chem.GetAdjacencyMatrix(mol)
    return np.array(adjacency)+np.eye(adjacency.shape[0])


def mol_features(smiles):
    try:
        mol = rdkit.Chem.MolFromSmiles(smiles)
    except:
        raise RuntimeError("SMILES cannot been parsed!")
    #mol = Chem.AddHs(mol)
    atom_feat = np.zeros((mol.GetNumAtoms(), num_atom_feat))
    for atom in mol.GetAtoms():
        atom_feat[atom.GetIdx(), :] = atom_features(atom)
    adj_matrix = adjacent_matrix(mol)
    return atom_feat, adj_matrix


In [ ]:
!pip install word2vec

In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb
import word2vec_
from word2vec_ import seq_to_kmers, get_protein_embedding

In [ ]:
if __name__ == "__main__":

    from gensim.models import Word2Vec
    import os
    """
    atom_feature ,adj = mol_features("C(C(C(=O)[O-])[O-])(C(=O)[O-])[O-].C(C(C(=O)[O-])[O-])(C(=O)[O-])[O-].[K+].[K+].[Sb+3].[Sb+3]")
    print(atom_feature)
    print(adj)
    """
    DATASET = "GPCR_train"
    with open("./data/GPCR_train.txt","r") as f:
        data_list = f.read().strip().split('\n')
    """Exclude data contains '.' in the SMILES format."""
    data_list = [d for d in data_list if '.' not in d.strip().split()[0]]
    N = len(data_list)

    compounds, adjacencies,proteins,interactions = [], [], [], []
    model = Word2Vec.load("word2vec_30.model")
    for no, data in enumerate(data_list):
        print('/'.join(map(str, [no + 1, N])))
        smiles, sequence, interaction = data.strip().split(" ")

        atom_feature, adj = mol_features(smiles)
        compounds.append(atom_feature)
        adjacencies.append(adj)

        interactions.append(np.array([float(interaction)]))

        protein_embedding = get_protein_embedding(model, seq_to_kmers(sequence))
        proteins.append(protein_embedding)
    dir_input = ('dataset/' + DATASET + '/word2vec_30/')
    os.makedirs(dir_input, exist_ok=True)
    # np.save(dir_input + 'compounds', compounds)
    # np.save(dir_input + 'adjacencies', adjacencies)
    # np.save(dir_input + 'proteins', proteins)
    # np.save(dir_input + 'interactions', interactions)
    print('The preprocess of ' + DATASET + ' dataset has finished!')


In [ ]:
# Example of one element from GPCR_train
smiles, sequence, interaction = data_list[0].strip().split(" ")

In [ ]:
print(smiles)
print(sequence)
print(interaction)

CC(=O)Nc1cc(Cl)cc2cc(C(=O)N3CCN(Cc4ccc(F)cc4)CC3C)oc12
METPNTTEDYDTTTEFDYGDATPCQKVNERAFGAQLLPPLYSLVFVIGLVGNILVVLVLVQYKRLKNMTSIYLLNLAISDLLFLFTLPFWIDYKLKDDWVFGDAMCKILSGFYYTGLYSEIFFIILLTIDRYLAIVHAVFALRARTVTFGVITSIIIWALAILASMPGLYFSKTQWEFTHHTCSLHFPHESLREWKLFQALKLNLFGLVLPLLVMIICYTGIIKILLRRPNEKKSKAVRLIFVIMIIFFLFWTPYNLTILISVFQDFLFTHECEQSRHLDLAVQVTEVIAYTHCCVNPVIYAFVGERFRKYLRQLFHRRVAVHLVKWLPFLSVDRLERVSSTSPSTGEHELSAGF
1


For each molecule passed, the atom_feature, adj are obtained

In [ ]:
# This is and example for a molecule
example = "C(C(C(=O)[O-])[O-])(C(=O)[O-])[O-].C(C(C(=O)[O-])[O-])(C(=O)[O-])[O-].[K+].[K+].[Sb+3].[Sb+3]"
mol =  Chem.MolFromSmiles(example)
number_of_atoms_in_molecule = mol.GetNumAtoms()

# Create atom_feat with the number of atoms in the molecule and the number of features available for that atom
num_atom_feat=34
atom_feat = np.zeros((mol.GetNumAtoms(), num_atom_feat))

# For each atom, extract its features and find its adjacent_matrix

# For its features, extract features including atom symbol(10),degree(7),formal charge,radical electrons,hybridization(6),aromatic(1),Chirality(3)
for atom in mol.GetAtoms():
    atom_feat[atom.GetIdx(), :] = atom_features(atom)

# For its adjacent matrix, The example of an adjacency matrix is shown in Fig. 6.1. It is a square matrix with the number of columns and rows equal
# to the number of atoms in the molecule. Positions corresponding to bonded atoms are marked with 1 (or sometimes by bond type), and all other entries are zero
# http://www.ccl.net/cca/documents/molecular-modeling/node3.html#:~:text=The%20example%20of%20an%20adjacency,all%20other%20entries%20are%20zero.
adjacency = Chem.GetAdjacencyMatrix(mol)
adj_matrix =  np.array(adjacency)+np.eye(adjacency.shape[0])